In [1]:
from __future__ import print_function

# Dataset
Format:
- each row is an example.
- The last column is the class(GC or Non-GC).
- The first three columns are features.

In [2]:
training_dataSSD1 = [
    [4, 3, 80, 'GC'],
    [14, 3, 142, 'Non-GC'],
    [10, 1, 147, 'Non-GC'],
    [2, 1, 150, 'Non-GC'],
    [3, 3, 60, 'GC'],
]
training_dataSSD2 = [
    [4, 1, 54, 'GC'],
    [14, 3, 20, 'Non-GC'],
    [10, 1, 20, 'Non-GC'],
    [0, 1, 150, 'Non-GC'],
    [3, 3, 60, 'GC'],
]
training_dataSSD3 = [
    [15, 3, 5, 'GC'],
    [4, 3, 142, 'Non-GC'],
    [10, 1, 147, 'Non-GC'],
    [20, 5, 14, 'Non-GC'],
    [4, 3, 200, 'GC'],
]

# Column labels.
- These are used only to print the tree.

In [3]:
header = ["feature1", "feature2", "feature3" ,"class"]

## CART Algorithem

In [4]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [5]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [6]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [7]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [8]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [9]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [10]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [11]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [12]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [13]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [14]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [15]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [16]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

## Construct a decision tree for each SSD

In [17]:
my_treeSSD1 = build_tree(training_dataSSD1)
print("SSD1")
print_tree(my_treeSSD1)
my_treeSSD2 = build_tree(training_dataSSD2)
print("\nSSD2")
print_tree(my_treeSSD2)
my_treeSSD3 = build_tree(training_dataSSD3)
print("\nSSD3")
print_tree(my_treeSSD3)

SSD1
Is feature3 >= 142?
--> True:
  Predict {'Non-GC': 3}
--> False:
  Predict {'GC': 2}

SSD2
Is feature3 >= 54?
--> True:
  Is feature3 >= 150?
  --> True:
    Predict {'Non-GC': 1}
  --> False:
    Predict {'GC': 2}
--> False:
  Predict {'Non-GC': 2}

SSD3
Is feature3 >= 14?
--> True:
  Is feature3 >= 200?
  --> True:
    Predict {'GC': 1}
  --> False:
    Predict {'Non-GC': 3}
--> False:
  Predict {'GC': 1}


## TestData
Format: the output of Iostat
- The first column is the SSD.
- The other columns are the features.

In [37]:
testData=[
    ["sd1", 100, 7, 142],
    ["sd2", 4, 3, 142],
    ["sd3", 10, 1, 147],
]

In [38]:
SSDClass={}
for row in testData:
    
    if row[0]=="sd1":
        SSDClass[row[0]]=print_leaf(classify(row[1::], my_treeSSD1))
    if row[0]=="sd2":
        SSDClass[row[0]]=print_leaf(classify(row[1::], my_treeSSD2))
    if row[0]=="sd3":
        SSDClass[row[0]]=print_leaf(classify(row[1::], my_treeSSD3))
SSDClass

{'sd1': {'Non-GC': '100%'}, 'sd2': {'GC': '100%'}, 'sd3': {'Non-GC': '100%'}}